# 🤗 Transformers Exploration

## Generate some text

Link: [GPT-2 Model Card](https://huggingface.co/openai-community/gpt2)

In [1]:
from transformers import pipeline, set_seed

In [2]:
generator = pipeline('text-generation', model='gpt2')
set_seed(42)

Device set to use mps:0


In [5]:
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Hello, I\'m a language model, a language model", says a voice with a very low level of awareness. "For example, with JavaScript and'},
 {'generated_text': "Hello, I'm a language model, not a language model, it's pretty obvious that we need more complexity to model languages. But it's interesting"},
 {'generated_text': "Hello, I'm a language model, and when you say something like:\n\n[T]hem, the value of a variable type should be"},
 {'generated_text': "Hello, I'm a language model, not a computer programmer. I'll say this, there's nothing I could to hide.\n\nThen the"},
 {'generated_text': "Hello, I'm a language model, so I'm not really prepared for you to ask me questions here. I have a lot to learn. I"}]

## Visualise the state dict

In [6]:
from transformers import GPT2LMHeadModel

# Load the smallest GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Access the state dictionary
state_dict = model.state_dict()

# Inspect the state dictionary (e.g., print keys or specific weights)
print("State Dictionary Keys:")
for key, val in state_dict.items():
    print(key, val.shape)

State Dictionary Keys:
transformer.wte.weight torch.Size([50257, 768])
transformer.wpe.weight torch.Size([1024, 768])
transformer.h.0.ln_1.weight torch.Size([768])
transformer.h.0.ln_1.bias torch.Size([768])
transformer.h.0.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias torch.Size([2304])
transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias torch.Size([768])
transformer.h.0.ln_2.weight torch.Size([768])
transformer.h.0.ln_2.bias torch.Size([768])
transformer.h.0.mlp.c_fc.weight torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias torch.Size([3072])
transformer.h.0.mlp.c_proj.weight torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias torch.Size([768])
transformer.h.1.ln_1.weight torch.Size([768])
transformer.h.1.ln_1.bias torch.Size([768])
transformer.h.1.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.1.attn.c_attn.bias torch.Size([2304])
transformer.h.1.attn.c_proj.weight torch.Size([768, 768])
transforme

## Now lets create the GPT class

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [30]:
from dataclasses import dataclass
@dataclass
class GPTConfig:
    block_size: int = 1024 # max sequence length
    vocab_size: int = 50257 # number of tokens: 50,000 BPE merges + 256 bytes tokens + 1 <|endoftext|> token
    n_layers: int = 12 # number of layers
    n_heads: int = 12 # number of heads
    emb_dim: int = 768 # embedding dimension

In [31]:
device= torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
device

device(type='mps')

In [32]:
class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()

        assert config.emb_dim % config.n_heads == 0

        self.c_attn = nn.Linear(config.emb_dim, 3 * config.emb_dim)
        self.c_proj = nn.Linear(config.emb_dim, config.emb_dim)
        self.register_buffer("bias",\
                              torch.tril(torch.ones(config.block_size, config.block_size)).\
                                view(1, 1, config.block_size, config.block_size))
        self.n_heads = config.n_heads
        
    def forward(self, x):
        B, T, C = x.shape

        qkv = self.c_attn(x) # (B, T, 3*C)
        q, k, v = torch.split(qkv, C, dim=2)
        q = q.view(B, T, self.n_heads, C//self.n_heads).transpose(1, 2)
        k = k.view(B, T, self.n_heads, C//self.n_heads).transpose(1, 2)
        v = v.view(B, T, self.n_heads, C//self.n_heads).transpose(1, 2) # (B, nH, T, H)

        attn = q @ k.transpose(-2, -1) * ((k.size(-1)) ** (-0.50)) # (B, nH, T, T)
        attn_mask = attn.masked_fill(self.bias[:, :, :T, :T]==0, float("-inf"))
        attn_scores = torch.softmax(attn_mask, dim=-1)

        out = attn_scores @ v # (B, nH, T, H)
        out = out.transpose(1, 2).contiguous().view(B, T, C)

        out = self.c_proj(out)

        return out

        
class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.emb_dim, 4 * config.emb_dim)
        self.gelu = nn.GELU(approximate="tanh")
        self.c_proj = nn.Linear(4 * config.emb_dim, config.emb_dim)

    def forward(self, x):
        # x shape: (B, T, C)
        x = self.c_fc(x) # (B, T, 4 * C)
        x = self.gelu(x)
        x = self.c_proj(x) # (B, T, C)
        return x


class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.emb_dim)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.emb_dim)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x)) # residual (B,T,C)
        x = x + self.mlp(self.ln_2(x)) # residual (B,T,C)
        return x


class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.emb_dim), #[50257, 768]
            wpe = nn.Embedding(config.block_size, config.emb_dim), #[1024, 768]
            h = nn.ModuleList([Block(config) for _ in range(config.n_layers)]),
            ln_f = nn.LayerNorm(config.emb_dim)
        ))

        self.lm_head = nn.Linear(config.emb_dim, config.vocab_size, bias=False)

        self.block_size = config.block_size

    def forward(self, x, targets=None):
        # x shape (B, T), targets shape (B, T)
        B, T =  x.shape
        token_emb = self.transformer.wte(x) # (B, T, C)
        pos_emb = self.transformer.wpe(torch.arange(0, T,\
                                                     device=device)) # (T, C)
        x = token_emb + pos_emb # (B, T, C)

        for layer in self.transformer.h:
            x = layer(x)

        x = self.transformer.ln_f(x)
        logits = self.lm_head(x) # (B, T, V)

        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, self.vocab_size), targets.view(-1)).item()

        return logits, loss
    
    @classmethod
    def from_pretrained(cls, model_type):
        """Loads pretrained GPT-2 model weights from huggingface"""
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layers=12, n_heads=12, emb_dim=768),  # 124M params
            'gpt2-medium':  dict(n_layers=24, n_heads=16, emb_dim=1024), # 350M params
            'gpt2-large':   dict(n_layers=36, n_heads=20, emb_dim=1280), # 774M params
            'gpt2-xl':      dict(n_layers=48, n_heads=25, emb_dim=1600), # 1558M params
        }[model_type]
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model

### Load from pretrained

In [33]:
config = GPTConfig()
config

GPTConfig(block_size=1024, vocab_size=50257, n_layers=12, n_heads=12, emb_dim=768)

In [34]:
model = GPT.from_pretrained(model_type="gpt2")

loading weights from pretrained gpt: gpt2


### Lets sample some text again from tis

In [35]:
model.to(device);
model.eval();

In [36]:
import tiktoken
tokenizer = tiktoken.encoding_for_model("gpt-2")
tokenizer.n_vocab

50257

In [39]:
@torch.no_grad()
def generate(model, text ,max_length, num_return_sequences, k=50):
    # first tokenize the text
    tokens = tokenizer.encode(text)
    tokens = torch.tensor(tokens, device=device).long()
    print(tokens)
    print("-"*50)
    # next make it (B, T)
    tokens = tokens.unsqueeze(0).expand(num_return_sequences, -1)

    for _ in range(max_length):
        x = tokens[:, -config.block_size:] # (B, T)
        print(x)
        logits, _ = model(x) # (B, T, V)

        logits = logits[:, -1, :] # (B, V)

        probs = F.softmax(logits, dim=-1)

        topk_probs, topk_indices = torch.topk(probs, k, dim=-1)

        ix = torch.multinomial(topk_probs, num_samples=1) # (B, 1)
        sampled_indices = torch.gather(topk_indices, -1, ix) # (B, 1)

        tokens = torch.cat((tokens, sampled_indices), dim=1)

    output_text = tokenizer.decode_batch(tokens.cpu().detach().numpy())
    return output_text

In [40]:
torch.manual_seed(42)
if device == "mps":
    torch.mps.manual_seed(42)
elif device == "cuda":
    torch.cuda.manual_seed(42)

In [41]:
generate(model, "Hello, I'm a language model," ,max_length=30, num_return_sequences=5)

tensor([15496,    11,   314,  1101,   257,  3303,  2746,    11],
       device='mps:0')
--------------------------------------------------
tensor([[15496,    11,   314,  1101,   257,  3303,  2746,    11],
        [15496,    11,   314,  1101,   257,  3303,  2746,    11],
        [15496,    11,   314,  1101,   257,  3303,  2746,    11],
        [15496,    11,   314,  1101,   257,  3303,  2746,    11],
        [15496,    11,   314,  1101,   257,  3303,  2746,    11]],
       device='mps:0')
tensor([[15496,    11,   314,  1101,   257,  3303,  2746,    11,   543],
        [15496,    11,   314,  1101,   257,  3303,  2746,    11,   290],
        [15496,    11,   314,  1101,   257,  3303,  2746,    11,   314],
        [15496,    11,   314,  1101,   257,  3303,  2746,    11,   290],
        [15496,    11,   314,  1101,   257,  3303,  2746,    11,   880]],
       device='mps:0')
tensor([[15496,    11,   314,  1101,   257,  3303,  2746,    11,   543,  1724],
        [15496,    11,   314,  1101,  

["Hello, I'm a language model, which means I'm familiar with it, but I'm not fluent in that. Well, with that said, I know how to put this together.",
 "Hello, I'm a language model, and the syntax, to make use of it, is pretty good. So why do you have that and not try using something else on it (from",
 "Hello, I'm a language model, I'm doing this work in Python, and then I'm writing code for Haskell.\n\nSo we can see, in this case, my previous",
 "Hello, I'm a language model, and you're making assumptions about my use of them. I'm not a natural language learner. I'm not a linguist. I'm interested",
 "Hello, I'm a language model, well, I'm from Java and have to write a programming language for it. I have my own vocabulary because I think of things differently for the first"]

In [229]:
generator = pipeline('text-generation', model='gpt2')
set_seed(42)

Device set to use mps:0


In [230]:
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm a language model, and my project will get better with time, but I think there are a lot more things that can help you"},
 {'generated_text': "Hello, I'm a language model, not a language model, so if I don't have a problem, I can fix it by creating new words"},
 {'generated_text': "Hello, I'm a language model, and I'm trying to learn some stuff. I'll try to do some basic programming and just learn better ways"},
 {'generated_text': "Hello, I'm a language model, but I don't believe in grammar. This will work for every language model. You can define it very quickly"},
 {'generated_text': 'Hello, I\'m a language model, a model of how things should be, and then we look at different things as well." I\'d like to'}]

In [7]:
import torch

In [8]:
x = torch.rand((3,4,5))
x

tensor([[[0.8823, 0.9150, 0.3829, 0.9593, 0.3904],
         [0.6009, 0.2566, 0.7936, 0.9408, 0.1332],
         [0.9346, 0.5936, 0.8694, 0.5677, 0.7411],
         [0.4294, 0.8854, 0.5739, 0.2666, 0.6274]],

        [[0.2696, 0.4414, 0.2969, 0.8317, 0.1053],
         [0.2695, 0.3588, 0.1994, 0.5472, 0.0062],
         [0.9516, 0.0753, 0.8860, 0.5832, 0.3376],
         [0.8090, 0.5779, 0.9040, 0.5547, 0.3423]],

        [[0.6343, 0.3644, 0.7104, 0.9464, 0.7890],
         [0.2814, 0.7886, 0.5895, 0.7539, 0.1952],
         [0.0050, 0.3068, 0.1165, 0.9103, 0.6440],
         [0.7071, 0.6581, 0.4913, 0.8913, 0.1447]]])

In [9]:
x.shape

torch.Size([3, 4, 5])

In [10]:
y = x[:, -1, :]
y.shape

torch.Size([3, 5])

In [11]:
logits = torch.randn((5, 200))

In [12]:
probs = torch.softmax(logits, dim=-1)

In [15]:
probs[2].sum()

tensor(1.)

In [16]:
topk_probs, topk_indices = torch.topk(probs, dim=-1, k=50) # (B, 50)
topk_probs.shape, topk_indices.shape

(torch.Size([5, 50]), torch.Size([5, 50]))

In [17]:
topk_probs[0]

tensor([0.1580, 0.0257, 0.0231, 0.0206, 0.0193, 0.0165, 0.0159, 0.0153, 0.0148,
        0.0141, 0.0139, 0.0126, 0.0123, 0.0120, 0.0119, 0.0118, 0.0117, 0.0112,
        0.0108, 0.0101, 0.0100, 0.0098, 0.0090, 0.0087, 0.0082, 0.0081, 0.0080,
        0.0077, 0.0073, 0.0073, 0.0073, 0.0070, 0.0066, 0.0066, 0.0062, 0.0061,
        0.0060, 0.0059, 0.0059, 0.0056, 0.0055, 0.0055, 0.0054, 0.0054, 0.0053,
        0.0053, 0.0053, 0.0051, 0.0050, 0.0050])

In [18]:
topk_probs[0].sum()

tensor(0.6419)

In [19]:
sampled_idx = torch.multinomial(topk_probs, num_samples=1, replacement=True) # (B,)
sampled_idx.shape

torch.Size([5, 1])

In [20]:
sampled_idx

tensor([[26],
        [19],
        [12],
        [ 1],
        [37]])

In [21]:
topk_indices

tensor([[ 58,   6, 141, 161,  68, 132,  40,  19, 165,  86,  95, 159,  80, 106,
          87,   4,  27,   2, 109, 110,  59, 188,   8,  76, 157, 144,  25,  69,
          93, 134,  85,  79,   5, 190, 123,  38,  63,  45,  28, 163, 105,  74,
         162,  99, 192, 184, 153,  31,  49, 194],
        [ 57, 184, 131,  35, 110,  36,  21, 100,  65,  64,  91, 111, 101,  82,
          85, 196, 112, 192, 175, 138, 162, 154,  98, 161,  59, 137,  20, 151,
         173,  87,  48,  62, 114, 150,  23,  28, 178, 148,  38, 106, 130,  25,
          76,  66,  42, 135,   8,  46, 176,  24],
        [107, 101, 149,  92, 182, 100, 191,  95, 157,  20, 158, 192, 175,  98,
           2, 113, 194,  68, 129, 104,   1, 105,  78,  41,  36, 110,   0,  94,
          87,   4,  49,  89, 160,  40,  96,  48, 152, 130,  50, 115,  21,  35,
          47, 174,  66,  72,  70, 134, 185, 126],
        [ 31,  25,  75,  12,  63,  87, 167, 102,  69, 113, 162,  40,   0, 175,
          46, 190,  91, 149,  38, 176, 153,  64, 130, 164, 1

In [23]:
sampled_indices = torch.gather(topk_indices, -1, sampled_idx) # (B, 1)
sampled_indices

tensor([[ 25],
        [138],
        [175],
        [ 25],
        [  3]])

In [24]:
sampled_indices.shape

torch.Size([5, 1])

In [26]:
[1,2,3,4,5][-3:]

[3, 4, 5]

In [42]:
x = torch.rand((3,4,5))
x

tensor([[[0.8823, 0.9150, 0.3829, 0.9593, 0.3904],
         [0.6009, 0.2566, 0.7936, 0.9408, 0.1332],
         [0.9346, 0.5936, 0.8694, 0.5677, 0.7411],
         [0.4294, 0.8854, 0.5739, 0.2666, 0.6274]],

        [[0.2696, 0.4414, 0.2969, 0.8317, 0.1053],
         [0.2695, 0.3588, 0.1994, 0.5472, 0.0062],
         [0.9516, 0.0753, 0.8860, 0.5832, 0.3376],
         [0.8090, 0.5779, 0.9040, 0.5547, 0.3423]],

        [[0.6343, 0.3644, 0.7104, 0.9464, 0.7890],
         [0.2814, 0.7886, 0.5895, 0.7539, 0.1952],
         [0.0050, 0.3068, 0.1165, 0.9103, 0.6440],
         [0.7071, 0.6581, 0.4913, 0.8913, 0.1447]]])

In [43]:
x.shape[-1]

5

In [45]:
x.size(-1)


5